# 线性分类实践

在这个实践中，我们将学习如何使用PyTorch实现线性分类模型。我们将以预测混凝土裂缝是否需要修复为例，这是土木工程中的一个常见问题。

## 1. 导入必要的库

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

## 2. 准备数据

在实际项目中，数据通常来自实地检查。这里，我们将创建一些模拟数据来演示这个过程。

假设我们有一组数据，包含裂缝宽度(mm)和裂缝深度(mm),以及是否需要修复(0表示不需要，1表示需要)。

In [ ]:
# 生成模拟数据
np.random.seed(42)
n_samples = 1000

width = np.random.uniform(0, 10, n_samples)
depth = np.random.uniform(0, 20, n_samples)
needs_repair = (width > 3) | (depth > 10)

X = np.column_stack((width, depth))
y = needs_repair.astype(int)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 转换为PyTorch张量
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.LongTensor(y_test)

# 可视化数据
plt.figure(figsize=(10, 8))
plt.scatter(X[y==0, 0], X[y==0, 1], label='no need to fix', alpha=0.5)
plt.scatter(X[y==1, 0], X[y==1, 1], label='need to fix', alpha=0.5)
plt.xlabel('crack width (mm)')
plt.ylabel('crack depth (mm)')
plt.title('concrete crack dataset')
plt.legend()
plt.show()

## 3. 定义数据集和数据加载器

In [ ]:
class CrackDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = CrackDataset(X_train, y_train)
test_dataset = CrackDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## 4. 定义模型

我们将使用PyTorch的`nn.Module`来定义我们的线性分类模型。

In [ ]:
class LinearClassifier(nn.Module):
    def __init__(self, input_dim):
        super(LinearClassifier, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

# 实例化模型
model = LinearClassifier(2)
print(model)

## 5. 定义损失函数和优化器

对于二分类问题，我们使用二元交叉熵(Binary Cross Entropy)作为损失函数。
优化器我们使用Adam。

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

## 6. 训练模型

In [ ]:
num_epochs = 100
losses = []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        # 前向传播
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch.float())
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    losses.append(avg_loss)
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# 绘制损失曲线
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

## 7. 评估模型

In [ ]:
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch).squeeze()
        predicted = (outputs > 0.5).float()
        y_pred.extend(predicted.numpy())
        y_true.extend(y_batch.numpy())

accuracy = accuracy_score(y_true, y_pred)
print(f'Test Accuracy: {accuracy:.4f}')

# 绘制混淆矩阵
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

## 8. 可视化决策边界

In [ ]:
# 生成网格点
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))

# 预测网格点的类别
Z = model(torch.FloatTensor(np.c_[xx.ravel(), yy.ravel()])).detach().numpy()
Z = Z.reshape(xx.shape)

# 绘制决策边界
plt.figure(figsize=(10, 8))
plt.contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.RdYlBu)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu, edgecolor='black')
plt.xlabel('crack width (mm)')
plt.ylabel('crack depth (mm)')
plt.title('decision boundary')
plt.show()

## 9. 使用模型进行预测

现在我们可以使用训练好的模型来预测新的数据。

In [ ]:
# 假设我们要预测一条宽度为2.5mm,深度为8mm的裂缝是否需要修复
new_crack = torch.FloatTensor([[2.5, 8.0]])
prediction = model(new_crack).item()
needs_repair = 'Yes' if prediction > 0.5 else 'No'
print(f'对于宽度为2.5mm,深度为8mm的裂缝:')
print(f'需要修复的概率: {prediction:.2f}')
print(f'是否需要修复: {needs_repair}')

## 10. 总结

在这个实践中，我们学习了如何使用PyTorch实现一个简单的线性分类模型来预测混凝土裂缝是否需要修复。主要步骤包括:

1. 准备和可视化数据
2. 定义数据集和数据加载器
3. 定义模型结构
4. 选择损失函数和优化器
5. 训练模型
6. 评估模型性能
7. 可视化决策边界
8. 使用模型进行预测

这个例子展示了如何将深度学习应用于土木工程中的实际问题。通过这种方法，我们可以建立一个模型来自动判断混凝土裂缝是否需要修复，这对于结构健康监测和维护决策都有重要意义。

在实际应用中，我们可能需要考虑更多的因素(如裂缝位置、结构重要性等),这时可以扩展到多特征分类或更复杂的非线性模型。同时，收集高质量的实际数据对于模型的准确性至关重要。